In [13]:
import pandas as pd
import numpy as np
#import gc

import mlcrate as mlc
import pickle as pkl
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Embedding, Dense, Flatten, Concatenate, 
                                     Dot, Reshape, Add, Subtract, BatchNormalization)
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

In [2]:
data_path = "/Users/macbook/Downloads/machine_learning_08_data/" 

In [3]:
train = pd.read_csv(data_path + 'train_sample.csv')
test = pd.read_csv(data_path + 'test_sample.csv')

In [4]:
features = ['feature_0', 'feature_1', 'feature_2']
f_size  = [int(train[f].max()) + 1 for f in features]
X_train = [train[f].values for f in features]
X_test = [test[f].values for f in features]
Y_train = [train['target'].values]

In [42]:
n_epochs = 300
batch_size = 128

In [43]:
k_latent = 2
embedding_reg = 0.0002 
kernel_reg = 0.1 

In [44]:
out_dim=2
def get_embed(x_input, x_size, out_dim,test_weights=False):
    # x_input is index of input (either user or item)
    # x_size is length of vocabulary (e.g. total number of users or items)
    # test_weights is a demo flag to show results with predefined weights
    # out_dim is size of embedding vectors
    if x_size > 0: #category
        if test_weights & (out_dim<=2):
            embed = Embedding(x_size, out_dim, input_length=1,
                          weights=[weight_matrix[:x_size,:out_dim]], 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        else:
            embed = Embedding(x_size, out_dim, input_length=1,
                              embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(out_dim, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

In [45]:
def build_model(f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
    
    lin_terms = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]

    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
     
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(lin_terms + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    model.compile(optimizer=Adam(clipnorm=0.25,learning_rate=0.001), 
                  loss='mean_squared_error')
    return model

In [46]:
model = build_model(f_size)

In [47]:
earlystopper = EarlyStopping(patience=2, verbose=1)
t = mlc.time.Timer()

In [48]:
model.fit(X_train,  Y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=0, shuffle=True,
          callbacks=[earlystopper])

In [49]:
predictions = model.predict(X_train)
#RMSE = sklearn.metrics.mean_squared_error(Y_train, predictions)
predictions_ = [[i[0] for i in predictions]]
RMSE = sklearn.metrics.mean_squared_error(Y_train, predictions_)
RMSE

282/282 [==============================] - 0s 894us/step


0.039440249015880356

In [50]:
predictions = model.predict(X_test)
predictions

32/32 [==============================] - 0s 969us/step


array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [3.0621538 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.7021809 ],
       [0.8684989 ],
       [0.16666427],
       [0.        ],
       [0.19703591],
       [0.62319565],
       [0.        ],
       [0.4776148 ],
       [0.        ],
       [2.6906385 ],
       [0.        ],
       [1.2502269 ],
       [4.7991066 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.8616894 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [2.8108752 ],
       [0.        ],
       [0.44072568],
       [0.6571261 ],
       [2.478995  ],
       [0.        ],
       [1.0613437 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.26553607],
       [2.1905632 ],
       [1.5434793 ],
       [1.4373981 ],
       [0.        ],
       [0.        ],
       [0.8334119 ],
       [0.   

In [51]:
sub = pd.read_csv(data_path + 'test_sample.csv',usecols=['ID'])
sub['target'] = predictions
sub.to_csv('submission.csv',index=False)